In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Путь к папкам с данными
train_path = 'C:/Users/Susanna/anaconda3/train'
test_path = 'C:/Users/Susanna/anaconda3/test'

# Загрузка данных из папок
def load_data(folder_path):
    data = []
    labels = []
    for label in ['neg', 'pos']:
        label_path = os.path.join(folder_path, label)
        for file_name in os.listdir(label_path):
            file_path = os.path.join(label_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                data.append(content)
                # Добавляем проверку наличия рейтинга в имени файла
                rating_start = file_name.find('_')
                rating_end = file_name.find('.txt')
                if rating_start != -1 and rating_end != -1:
                    rating = int(file_name[rating_start+1:rating_end])
                    labels.append({'content': content, 'rating': rating, 'sentiment': label})
                else:
                    labels.append(None)  # Если рейтинг отсутствует, можно указать None
    return labels

# Загрузка данных из папок
train_data = load_data(train_path)
test_data = load_data(test_path)

# Фильтрация данных без рейтинга
train_data = [item for item in train_data if item is not None]
test_data = [item for item in test_data if item is not None]

# Создание DataFrame для удобства работы с данными
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Создание модели для предсказания рейтинга
model = make_pipeline(
    TfidfVectorizer(), 
    StandardScaler(with_mean=False),
    LinearRegression()
)

# Обучение модели
model.fit(train_df['content'], train_df['rating'])

# Предсказание рейтинга для тестовых данных
ratings_pred = model.predict(test_df['content'])

# Оценка качества модели
mse = mean_squared_error(test_df['rating'], ratings_pred)

# Сохранение модели
import joblib
joblib.dump(model, 'regression_model.joblib')

['regression_model.joblib']